### Computing PCA using RDDs

###  PCA

The vectors that we want to analyze have length, or dimension, of 365, corresponding to the number of 
days in a year.

We want to perform [Principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)
on these vectors. There are two steps to this process:

1. Computing the covariance matrix: this is a  simple computation. However, it takes a long time to compute and it benefits from using an RDD because it involves all of the input vectors.
2. Computing the eigenvector decomposition. this is a more complex computation, but it takes a fraction of a second because the size to the covariance matrix is $365 \times 365$, which is quite small. We do it on the head node usin `linalg`

### Computing the covariance matrix
Suppose that the data vectors are the column vectors denoted $x$ then the covariance matrix is defined to be
$$
E(x x^T)-E(x)E(x)^T
$$

Where $x x^T$ is the **outer product** of $x$ with itself.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

### `nan`s in arithmetic operations
* We store all of the measurements as single `bytearray` in a single column. Instead of using 365 columns.
* Why?
  1. Because serializing and desirializing is faster that way.
  1. Because numpy treats `nan` entries correctly:
      * In `numpy.nansum` `5+nan=5` while in dataframes `5+nan=nan`

In [1]:
import numpy as np
X=np.array([1,1,1,2])
print 'mean of',X,'=',np.mean(X)
print 'nanmean of',X,'=',np.nanmean(X)
X=np.array([1,1,np.NaN,2])
print 'mean of',X,'=',np.mean(X)
print 'nanmean of',X,'=',np.nanmean(X)

mean of [1 1 1 2] = 1.25
nanmean of [1 1 1 2] = 1.25
mean of [  1.   1.  nan   2.] = nan
nanmean of [  1.   1.  nan   2.] = 1.33333333333


#### When should you not use `np.nanmean` ?
Using `n.nanmean` is equivalent to assuming that choice of which elements to remove is independent of the values of the elements. 
* Example of bad case: suppose the larger elements have a higher probability of being `nan`. In that case `np.nanmean` will under-estimate the mean

#### Computing Cov matrix on vectors with NaNs
As it happens, we often get vectors $x$ in which some, but not all, of the entries are `nan`. 
Suppose that we want to compute the mean of the elements of $x$. If we use `np.mean` we will get the result `nan`. A useful alternative is to use `np.nanmean` which removes the `nan` elements and takes the mean of the rest.

#### Computing the covariance  when there are `nan`s
The covariance is a mean of outer products.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

In [2]:
x1=np.array([1,np.NaN,3,4,5])
x2=np.array([2,3,4,np.NaN,6])
stacked=np.array([np.outer(x1,x1),np.outer(x2,x2)])
stacked

array([[[  1.,  nan,   3.,   4.,   5.],
        [ nan,  nan,  nan,  nan,  nan],
        [  3.,  nan,   9.,  12.,  15.],
        [  4.,  nan,  12.,  16.,  20.],
        [  5.,  nan,  15.,  20.,  25.]],

       [[  4.,   6.,   8.,  nan,  12.],
        [  6.,   9.,  12.,  nan,  18.],
        [  8.,  12.,  16.,  nan,  24.],
        [ nan,  nan,  nan,  nan,  nan],
        [ 12.,  18.,  24.,  nan,  36.]]])

In [3]:
np.nanmean(stacked,axis=0)

/home/vaibhav/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


array([[  2.5,   6. ,   5.5,   4. ,   8.5],
       [  6. ,   9. ,  12. ,   nan,  18. ],
       [  5.5,  12. ,  12.5,  12. ,  19.5],
       [  4. ,   nan,  12. ,  16. ,  20. ],
       [  8.5,  18. ,  19.5,  20. ,  30.5]])

### Loading Data into Dataframe

In [4]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
## add after creating spark_PCA.py
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/computeStats.py']) #,'lib/spark_PCA.py'

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [5]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
# add after creating spark_PCA.py
# from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Climate data

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

There is a large variety of measurements from all over the world, from 1870 will 2012.
in the directory `../../Data/Weather` you will find the following useful files:

* data-source.txt: the source of the data
* ghcnd-readme.txt: A description of the content and format of the data
* ghcnd-stations.txt: A table describing the Meteorological stations.



### Data cleaning

* Most measurements exists only for a tiny fraction of the stations and years. We therefor restrict our use to the following measurements:
```python
['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
```

* We consider only measurement-years that have at most 50 `NaN` entries

* We consider only measurements in the continential USA

* We partition the stations into 256 geographical rectangles, indexed from BBBBBBBB to SSSSSSSS. And each containing about 12,000 station,year pairs.

In [6]:
file_index='SBSBBSSB'
filename='US_Weather_%s.csv.gz'%file_index
data_dir='../../Data/Weather'
del_files='%s/%s*'%(data_dir,filename[:-3])
print del_files
!rm $del_files
command="curl https://mas-dse-open.s3.amazonaws.com/Weather/small/%s > %s/%s"%(filename,data_dir,filename)
print command
!$command
!ls -lh $data_dir/$filename

../../Data/Weather/US_Weather_SBSBBSSB.csv*
rm: cannot remove '../../Data/Weather/US_Weather_SBSBBSSB.csv*': No such file or directory
curl https://mas-dse-open.s3.amazonaws.com/Weather/small/US_Weather_SBSBBSSB.csv.gz > ../../Data/Weather/US_Weather_SBSBBSSB.csv.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3492k  100 3492k    0     0  1431k      0  0:00:02  0:00:02 --:--:-- 1446k
-rw-rw-r-- 1 vaibhav vaibhav 3.5M May 13 13:56 ../../Data/Weather/US_Weather_SBSBBSSB.csv.gz


In [9]:
!gunzip $data_dir/$filename
filename=data_dir+'/US_Weather_SBSBBSSB.csv'
!ls -lh $filename

-rw-rw-r-- 1 vaibhav vaibhav 13M May 13 13:56 ../../Data/Weather/US_Weather_SBSBBSSB.csv


In [10]:
import pickle
List=pickle.load(open(filename,'rb'))
len(List)

12655

In [11]:
df=sqlContext.createDataFrame(List)
print df.count()
df.show(5)

12655
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    124.1| 46.2667|-119.1167|       TMAX|USW00024163|    36|[00 7E 00 7E 00 7...|1998.0|SBSBBSSB|
|    124.1| 46.2667|-119.1167|       TMAX|USW00024163|     1|[80 54 30 54 40 C...|1999.0|SBSBBSSB|
|    124.1| 46.2667|-119.1167|       TMAX|USW00024163|     3|[30 55 E0 55 80 5...|2000.0|SBSBBSSB|
|    124.1| 46.2667|-119.1167|       TMAX|USW00024163|     3|[20 50 80 51 00 4...|2001.0|SBSBBSSB|
|    124.1| 46.2667|-119.1167|       TMAX|USW00024163|     1|[E0 50 80 51 30 5...|2002.0|SBSBBSSB|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
only showing top 5 rows



In [12]:
sqlContext.registerDataFrameAsTable(df,'weather') #using older sqlContext instead of newer (V2.0) sparkSession

In [13]:
from time import time
t=time()

N=sc.defaultParallelism
print 'Number of executors=',N
print 'took',time()-t,'seconds'

Number of executors= 3
took 0.00241804122925 seconds


### Computing PCA for each measurement

In [14]:
measurements=['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']

#### Homework
The code below computes the covariance matrix using RDDs.
The code allows undefined entries and calculates the covariance without bias.

Your homework is to complete the missing parts in the code (Marked with `...`) so that it would calculate the covariance correctly.

In [16]:
#%%writefile lib/spark_PCA.py #once this works correctly, you should add it to the `lib` directory
# and use import

import numpy as np
from numpy import linalg as LA

def outerProduct(X):
    """Computer outer product and indicate which locations in matrix are undefined"""
    O=np.outer(X,X)
    N=1-np.isnan(O)
    return (O,N)

def sumWithNan(M1,M2):
    """Add two pairs of (matrix,count)"""
    (X1,N1)=M1
    (X2,N2)=M2
    N=N1+N2
    X=np.nansum(np.dstack((X1,X2)),axis=2)
    return (X,N)

def computeCov(RDDin):
    # input: RDDin is an RDD of np arrays, all of the same length
    
    # we insert 1 at the beginning of each vector so the calculation also yields the mean vector
    RDD=RDDin.map(lambda v:np.array(np.insert(v,0,1),dtype=np.float64)) 
    #RDD=RDDin.map(lambda v:np.insert(v,0,1)) 
    
    #separating map and reduce does not matter, since Spark uses lazy execution.
    OuterRDD=RDD.map(outerProduct)    #<-- do mapping here
    (S,N)=OuterRDD.reduce(sumWithNan)  #<-- do reducing here
    print type(S), S.shape
    print type(N), N.shape
    E=S[0,1:]
    NE=np.float64(N[0,1:])
    print 'shape of E=',E.shape,'shape of NE=',NE.shape
    Mean=E/NE
    print Mean
    O=S[1:,1:]
    NO=np.float64(N[1:,1:])
    ex = outerProduct(Mean)
    Cov=  np.divide(O,NO) - ex[0]# This is the covariance matrix
    
    # Output also the diagnal which is the variance for each day
    Var=np.array([Cov[i,i] for i in range(Cov.shape[0])])
    return {'E':E,'NE':NE,'O':O,'NO':NO,'Cov':Cov,'Mean':Mean,'Var':Var}

In [18]:
from numpy import linalg as LA
STAT={}  # dictionary storing the statistics for each measurement
Clean_Tables={}

for meas in measurements:
    t=time()
    Query="SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
    print Query
    df = sqlContext.sql(Query)
    data=df.rdd.map(lambda row: unpackArray(row['vector'],np.float16))
    #get very basic statistics
    STAT[meas]=computeOverAllDist(data)   # Compute the statistics 

    # compute covariance matrix
    OUT=computeCov(data)

    #find PCA decomposition
    eigval,eigvec=LA.eig(OUT['Cov'])

    # collect all of the statistics in STAT[meas]
    STAT[meas]['eigval']=eigval
    STAT[meas]['eigvec']=eigvec
    STAT[meas].update(OUT)

    print 'time for',meas,'is',time()-t

SELECT * FROM weather
	WHERE measurement = 'TMAX'
<type 'numpy.ndarray'> (366, 366)
<type 'numpy.ndarray'> (366, 366)
shape of E= (365,) shape of NE= (365,)
[  20.76136364   18.50365591   15.12755973   15.96165317   18.43755346
   19.7022835    23.85406802   24.91709622   25.37644046   26.2156611
   25.0293483    25.30253545   25.67133758   31.06771721   31.06855692
   28.04885302   29.42338882   30.72493573   28.01614274   27.01197605
   28.31109222   28.45711872   32.02292994   33.81016949   34.81643132
   31.48380222   31.13518124   32.55888651   33.06255356   36.66264541
   34.95151515   36.85946177   38.87397849   43.83461538   48.7182087
   47.51465662   48.20008389   49.81711409   50.47333053   49.69475891
   52.58673895   55.53419516   57.55866388   56.68021794   55.75755034
   58.53607383   62.18014244   63.65284757   62.70674487   66.49165275
   69.7406015    73.0740276    72.80584551   74.67208333   76.06419341
   74.32609603   73.84298209   76.88100209   80.57477026   82.25

In [19]:
from pickle import dump
filename=data_dir+'/STAT_%s.pickle'%file_index
dump((STAT,STAT_Descriptions),open(filename,'wb'))
!ls -lrth $data_dir

total 199M
-rw-rw-r-- 1 vaibhav vaibhav  79M May 11 10:03 STAT_BBBSBBBB.pickle
-rw-rw-r-- 1 vaibhav vaibhav  13M May 11 10:03 US_Weather_BBBSBBBB.csv
drwxrwxr-x 2 vaibhav vaibhav 4.0K May 11 10:03 US_Weather_BBBSBBBB.parquet
-rw-rw-r-- 1 vaibhav vaibhav  218 May 11 10:03 data-source.txt
drwxrwxr-x 2 vaibhav vaibhav  44K May 11 10:03 decon_BBBSBBBB.parquet
drwxrwxr-x 2 vaibhav vaibhav  44K May 11 10:03 decon_BBBSBBBB_SNWD.parquet
-rw-rw-r-- 1 vaibhav vaibhav  22K May 11 10:03 ghcnd-readme.txt
drwxrwxr-x 2 vaibhav vaibhav  44K May 11 10:03 decon_BBBSBBBB_TOBS.parquet
-rw-rw-r-- 1 vaibhav vaibhav 8.0M May 11 10:03 ghcnd-stations.txt
-rw-rw-r-- 1 vaibhav vaibhav  270 May 11 10:03 ghcnd-version.txt
-rw-rw-r-- 1 vaibhav vaibhav 7.5M May 11 10:03 ghcnd-stations_buffered.txt
-rw-rw-r-- 1 vaibhav vaibhav 2.1M May 11 10:03 stations.pkl.gz
-rw-rw-r-- 1 vaibhav vaibhav  13M May 13 13:56 US_Weather_SBSBBSSB.csv
-rw-rw-r-- 1 vaibhav vaibhav  78M May 13 14:03 STAT_SBSBBSSB.pickle
